## 1. Elaboração do problema

## 2. Importando as biblioteca e carregandos os dados

## 3. Contato inicial com os dados

## 4. Análise Exploratória dos Dados (EDA)

### 4.1 Correlações

## 5. Pré-processamento dos dados

## 6. Aplicando diversos modelos

## 7. Avaliando _Recursive Feature Elimination using Cross-Validation_ 

## 8. Tunagem dos hiperparâmetros

## 9. Modelo final

## 10. Considerações finais